<a href="https://colab.research.google.com/github/WilliamAshbee/splineexample/blob/main/InvertedAutoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import packages
import os
import torch 
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
 
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.utils import save_image

In [ ]:
#https://debuggercafe.com/implementing-deep-autoencoder-in-pytorch/

In [ ]:
# constants
NUM_EPOCHS = 10
LEARNING_RATE = 1e-3
BATCH_SIZE = 128

# image transformations
transform = transforms.Compose([
    transforms.ToTensor(),
])

In [ ]:
trainset = datasets.FashionMNIST(
    root='./data',
    train=True, 
    download=True,
    transform=transform
)
testset = datasets.FashionMNIST(
    root='./data',
    train=False,
    download=True,
    transform=transform
)
trainloader = DataLoader(
    trainset, 
    batch_size=BATCH_SIZE,
    shuffle=True
)
testloader = DataLoader(
    testset, 
    batch_size=BATCH_SIZE, 
    shuffle=True
)

In [ ]:
# utility functions
def get_device():
    if torch.cuda.is_available():
        device = 'cuda:0'
    else:
        device = 'cpu'
    return device
def make_dir():
    image_dir = 'FashionMNIST_Images'
    if not os.path.exists(image_dir):
        os.makedirs(image_dir)
def save_decoded_image(img, epoch):
    img = img.view(img.size(0), 1, 28, 28)
    save_image(img, './FashionMNIST_Images/linear_ae_image{}.png'.format(epoch))

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        # encoder
        ins = 784
        self.enc1 = nn.Linear(in_features=ins, out_features=ins*3)
        self.enc2 = nn.Linear(in_features=ins*3, out_features=ins*6)
        self.enc3 = nn.Linear(in_features=ins*6, out_features=ins*9)
        self.enc4 = nn.Linear(in_features=ins*9, out_features=ins*12)
        self.enc5 = nn.Linear(in_features=ins*12, out_features=ins*15)

        # decoder 
        self.dec1 = nn.Linear(in_features=ins*15, out_features=ins*12)
        self.dec2 = nn.Linear(in_features=ins*12, out_features=ins*9)
        self.dec3 = nn.Linear(in_features=ins*9, out_features=ins*6)
        self.dec4 = nn.Linear(in_features=ins*6, out_features=ins*3)
        self.dec5 = nn.Linear(in_features=ins*3, out_features=ins)

    def forward(self, x):
        x = F.relu(self.enc1(x))
        x = F.relu(self.enc2(x))
        x = F.relu(self.enc3(x))
        x = F.relu(self.enc4(x))
        x = F.relu(self.enc5(x))

        x = F.relu(self.dec1(x))
        x = F.relu(self.dec2(x))
        x = F.relu(self.dec3(x))
        x = F.relu(self.dec4(x))
        x = F.relu(self.dec5(x))
        return x

net = Autoencoder()
print(net)


Autoencoder(
  (enc1): Linear(in_features=784, out_features=2352, bias=True)
  (enc2): Linear(in_features=2352, out_features=4704, bias=True)
  (enc3): Linear(in_features=4704, out_features=7056, bias=True)
  (enc4): Linear(in_features=7056, out_features=9408, bias=True)
  (enc5): Linear(in_features=9408, out_features=11760, bias=True)
  (dec1): Linear(in_features=11760, out_features=9408, bias=True)
  (dec2): Linear(in_features=9408, out_features=7056, bias=True)
  (dec3): Linear(in_features=7056, out_features=4704, bias=True)
  (dec4): Linear(in_features=4704, out_features=2352, bias=True)
  (dec5): Linear(in_features=2352, out_features=784, bias=True)
)


In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)


In [ ]:
def train(net, trainloader, NUM_EPOCHS):
    train_loss = []
    for epoch in range(NUM_EPOCHS):
        running_loss = 0.0
        for data in trainloader:
            img, _ = data
            img = img.to(device)
            img = img.view(img.size(0), -1)
            optimizer.zero_grad()
            outputs = net(img)
            loss = criterion(outputs, img)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        loss = running_loss / len(trainloader)
        train_loss.append(loss)
        print('Epoch {} of {}, Train Loss: {:.3f}'.format(
            epoch+1, NUM_EPOCHS, loss))

        if epoch % 5 == 0:
            save_decoded_image(outputs.cpu().data, epoch)

    return train_loss

def test_image_reconstruction(net, testloader):
     for batch in testloader:
        img, _ = batch
        img = img.to(device)
        img = img.view(img.size(0), -1)
        outputs = net(img)
        outputs = outputs.view(outputs.size(0), 1, 28, 28).cpu().data
        save_image(outputs, 'fashionmnist_reconstruction.png')
        break


In [ ]:
# get the computation device
device = torch.device("cuda")
#get_device()
print(device)
# load the neural network onto the device
net.to(device)
make_dir()
# train the network
train_loss = train(net, trainloader, NUM_EPOCHS)
plt.figure()
plt.plot(train_loss)
plt.title('Train Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.savefig('deep_ae_fashionmnist_loss.png')
# test the network
test_image_reconstruction(net, testloader)

cuda
Epoch 1 of 10, Train Loss: 0.174
Epoch 2 of 10, Train Loss: 0.050
Epoch 3 of 10, Train Loss: 0.040
Epoch 4 of 10, Train Loss: 0.035
Epoch 5 of 10, Train Loss: 0.033
Epoch 6 of 10, Train Loss: 0.032
Epoch 7 of 10, Train Loss: 0.031


KeyboardInterrupt: ignored